In [ ]:
import os
from dotenv import load_dotenv

load_dotenv()
print("Upstage API Key:", os.getenv("UPSTAGE_API_KEY")[:5] + "*****")


Upstage API Key: up_Eu*****


In [ ]:
from langchain_community.document_loaders import PyPDFLoader

pdf_path = "../data/콘텐츠분쟁해결_사례.pdf"   
loader = PyPDFLoader(pdf_path)
docs = loader.load()

print(f"총 {len(docs)} 페이지 로드 완료")


c:\Users\user\AppData\Local\pypoetry\Cache\virtualenvs\mylangchain-app-SBe-Yh6W-py3.12\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


총 109 페이지 로드 완료


In [ ]:
from langchain_text_splitters import RecursiveCharacterTextSplitter

text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=1500,
    chunk_overlap=300,
    separators=[
        "\n【사건개요】",
        "\n【쟁점사항】",
        "\n【처리경위】",
        "\n【처리결과】",
        "\n■", "\n\n", "\n", ".", " ", ""
    ]
)

splits = text_splitter.split_documents(docs)
print(f"총 분할 청크 수: {len(splits)}")
print(splits[0].page_content[:300])  


총 분할 청크 수: 104
콘텐츠분쟁조정 법리 연구 2부
- 타 분쟁조정사례 조사 -


In [4]:
from langchain_upstage import UpstageEmbeddings
from langchain_community.vectorstores import FAISS

embeddings = UpstageEmbeddings(model="solar-embedding-1-large")
vectorstore = FAISS.from_documents(splits, embeddings)

print("벡터스토어 생성 완료")


벡터스토어 생성 완료


In [5]:
from langchain_upstage import ChatUpstage

retriever = vectorstore.as_retriever(
    search_type="similarity",
    search_kwargs={"k": 5}
)

llm = ChatUpstage(model="solar-pro", temperature=0.2)

print("Retriever와 LLM 설정 완료")


Retriever와 LLM 설정 완료


In [6]:
from langchain.prompts import PromptTemplate
from langchain.chains import RetrievalQA

prompt_template = """
당신은 콘텐츠 분야 전문 법률 자문가입니다. 
아래 분쟁조정 사례들을 바탕으로 정확하고 전문적인 법률 조언을 제공해주세요.

관련 분쟁사례:
{context}

상담 내용: {question}

답변 가이드라인:
1. 제시된 사례들을 근거로 답변하세요.
2. 관련 법령이나 조항이 있다면 명시하세요.
3. 비슷한 사례의 처리경위와 결과를 참고하세요.
4. 실무적 해결방안을 단계별로 제시하세요.
5. 사례에 없는 내용은 "제시된 사례집에서는 확인할 수 없습니다"라고 명시하세요.

전문 법률 조언:
"""

PROMPT = PromptTemplate(
    template=prompt_template,
    input_variables=["context", "question"],
)

qa_chain = RetrievalQA.from_chain_type(
    llm=llm,
    chain_type="stuff",
    retriever=retriever,
    chain_type_kwargs={"prompt": PROMPT},
    return_source_documents=True
)


In [7]:
test_questions = [
    "온라인 게임에서 시스템 오류로 아이템이 사라졌는데, 게임회사가 복구를 거부하고 있습니다. 어떻게 해결할 수 있나요?",
    "인터넷 강의를 중도 해지하려고 하는데 과도한 위약금을 요구받고 있습니다. 정당한가요?",
    "무료체험 후 자동으로 유료전환되어 요금이 청구되었습니다. 환불 가능한가요?",
    "미성년자가 부모 동의 없이 게임 아이템을 구매했습니다. 환불받을 수 있는 방법이 있나요?",
    "온라인 교육 서비스가 광고와 다르게 제공되어 계약을 해지하고 싶습니다. 가능한가요?"
]

for q in test_questions:
    print("="*80)
    print("질문:", q)
    result = qa_chain.invoke({"query": q})
    print("답변:", result["result"][:800], "...\n")


질문: 온라인 게임에서 시스템 오류로 아이템이 사라졌는데, 게임회사가 복구를 거부하고 있습니다. 어떻게 해결할 수 있나요?
답변: ### 전문 법률 조언: 온라인 게임 시스템 오류로 인한 아이템 복구 거부 사례 해결 방안  

#### 1. **사례 분석을 통한 법적 쟁점 정리**  
제시된 사례집에 따르면, 시스템 오류로 인한 아이템 복구 문제는 다음과 같은 쟁점으로 구분됩니다:  
- **(1) 시스템 오류의 객관적 입증**: 게임사의 기술적 결함인지, 이용자 측의 문제(예: PC/통신 환경)인지 확인 필요 (2009년 사례 참조).  
- **(2) 계정 소유권 및 약관 준수 여부**: 계정 명의자와 실소유주가 다를 경우, 게임사는 복구 요청을 거절할 수 있음 (2006년 사례 참조).  
- **(3) 게임사의 복구 의무**: 프로그램 오류가 인정될 경우, 게임사는 복구 의무가 있음 (2006년 프로그램 오류 사례 참조).  
- **(4) 아이템의 법적 성격**: 게임 아이템은 일반적으로 "전자적 데이터"로 분류되며, 민법상 재산권 적용이 제한적일 수 있음 (2007년 해킹 아이템 사례 참조).  

---

#### 2. **관련 법령 및 약관 검토**  
- **민법 제250조(도품·유실물 특례)**:  
  - 게임 아이템이 "금전"이 아닌 경우, 피해자(유실자)는 2년 내 반환 청구가 가능하나, **게임사가 아이템 소유권을 약관으로 보유**한 경우 적용이 어려울 수 있음.  
- **게임산업진흥에 관한 법률 제12조(게임물의 내용·이용 등)**:  
  - 게임사는 이용자에게 시스템 오류 발생 시 **신속한 조치 및 공지 의무**가 있음.  
- **게임사 이용약관**:  
  - 대부분의 게임사는 "아이템 소유권"을 보 ...

질문: 인터넷 강의를 중도 해지하려고 하는데 과도한 위약금을 요구받고 있습니다. 정당한가요?
답변: ### 전문 법률 조언: 인터넷 강의 중도 해지 시 과도한 위약금 요구의 적법성 검토  

#### 1. **사례 분